In [ ]:
# p
# 1 #include <iostream>
# 2 #include <opencv2/opencv.hpp>
# 3 #include <Eigen/Core>
# 4 #include <Eigen/Dense>
# 5
# 6 using namespace std;
# 7 using namespace Eigen;
# 8
# 9 int main(int argc, char ∗∗argv) {
# 10 double ar = 1.0, br = 2.0, cr = 1.0; // ground−truth values
# 11 double ae = 2.0, be = −1.0, ce = 5.0; // initial estimation
# 12 int N = 100; // num of data points
# 13 double w_sigma = 1.0; // sigma of the noise
# 14 double inv_sigma = 1.0 / w_sigma;
# 15 cv::RNG rng; // Random number generator
# 16
# 17 vector<double> x_data, y_data; // the data
# 18 for (int i = 0; i < N; i++) {
# 19 double x = i / 100.0;
# 20 x_data.push_back(x);
# 21 y_data.push_back(exp(ar ∗ x ∗ x + br ∗ x + cr) + rng.gaussian(w_sigma ∗
# w_sigma));
# 22 }
# 23
# 24 // start Gauss−Newton iterations
# 25 int iterations = 100;
# 26 double cost = 0, lastCost = 0;
# 27
# 28 chrono::steady_clock::time_point t1 = chrono::steady_clock::n
# 5.3. PRACTICE: CURVE FITTING 111
# 29 for (int iter = 0; iter < iterations; iter++) {
# 30 Matrix3d H = Matrix3d::Zero(); // Hessian = J^T W^{−1} J in Gauss−Newton
# 31 Vector3d b = Vector3d::Zero(); // bias
# 32 cost = 0;
# 33
# 34 for (int i = 0; i < N; i++) {
# 35 double xi = x_data[i], yi = y_data[i]; // the i−th data
# 36 double error = yi − exp(ae ∗ xi ∗ xi + be ∗ xi + ce);
# 37 Vector3d J; // jacobian
# 38 J[0] = −xi ∗ xi ∗ exp(ae ∗ xi ∗ xi + be ∗ xi + ce); // de/da
# 39 J[1] = −xi ∗ exp(ae ∗ xi ∗ xi + be ∗ xi + ce); // de/db
# 40 J[2] = −exp(ae ∗ xi ∗ xi + be ∗ xi + ce); // de/dc
# 41
# 42 H += inv_sigma ∗ inv_sigma ∗ J ∗ J.transpose();
# 43 b += −inv_sigma ∗ inv_sigma ∗ error ∗ J;
# 44
# 45 cost += error ∗ error;
# 46 }
# 47
# 48 // solve Hx=b
# 49 Vector3d dx = H.ldlt().solve(b);
# 50 if (isnan(dx[0])) {
# 51 cout << "result is nan!" << endl;
# 52 break;
# 53 }
# 54
# 55 if (iter > 0 && cost >= lastCost) {
# 56 cout << "cost: " << cost << ">= last cost: " << lastCost << ", break." <<
# endl;
# 57 break;
# 58 }
# 59
# 60 ae += dx[0];
# 61 be += dx[1];
# 62 ce += dx[2];
# 63
# 64 lastCost = cost;
# 65
# 66 cout << "total cost: " << cost << ", \t\tupdate: " << dx.transpose() <<
# 67 "\t\testimated params: " << ae << "," << be << "," << ce << endl;
# 68 }
# 69
# 70 chrono::steady_clock::time_point t2 = chrono::steady_clock::now();
# 71 chrono::duration<double> time_used = chrono::duration_cast<chrono::duration<double
# >>(t2 − t1);
# 72 cout << "solve time cost = " << time_used.count() << " seconds. " << endl;
# 73 cout << "estimated abc = " << ae << ", " << be << ", " << ce << endl;
# 74 return 0;
# 75 }